In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#os.environ["CUDA_VISIBLE_DEVICES"]=""
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import datetime

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%load_ext autoreload
%autoreload 2

from dataset import Data
from callback import Save

from transformations import chargenet_trafo

In [2]:
data = Data(dirs=['/home/iwsatlas1/peller/work/oscNext/level3_v01.03/140000_i3cols','/home/iwsatlas1/peller/work/oscNext/level3_v01.03/120000_i3cols'])

In [3]:
charge_input = tf.keras.Input(shape=(1,))
params_input = tf.keras.Input(shape=(len(data.labels),))
t = chargenet_trafo(labels=data.labels)

h = t(charge_input, params_input)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(1024, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(2048, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(1024, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(512, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(256, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(128, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(64, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
h = tf.keras.layers.Dense(32, activation="relu")(h)
h = tf.keras.layers.Dropout(0.001)(h)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(h)

charge_net = tf.keras.Model(inputs=[charge_input, params_input], outputs=outputs)
charge_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 8)]          0                                            
__________________________________________________________________________________________________
chargenet_trafo (chargenet_traf (None, 9)            0           input_1[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           320         chargenet_trafo[0][0]            
______________________________________________________________________________________________

In [4]:
optimizer = tf.keras.optimizers.Adam(0.0001)
charge_net.compile(loss='binary_crossentropy', optimizer=optimizer)

In [5]:
train_id = 'ChargeNet_2048_' + datetime.datetime.now().strftime("%d_%b_%Y-%Hh%M")

In [6]:
callbacks = []
callbacks.append(Save(save_every=25, path_template='models/'+train_id+'/epoch_%i', ))
callbacks.append(tf.keras.callbacks.TensorBoard(log_dir='logs/'+train_id, histogram_freq=1))

In [7]:
train_data, test_data = data.get_chargenet_data(train_batch_size=4096,
                                                test_batch_size=4096,
                                                )

In [ ]:
hist = charge_net.fit(train_data, epochs=1000, verbose=2, callbacks=callbacks, validation_data=test_data,
                     initial_epoch=len(callbacks[0].hist['train_losses']))

Epoch 1/1000
988/988 - 18s - loss: 0.6143 - val_loss: 0.5836
Epoch 2/1000
988/988 - 19s - loss: 0.5773 - val_loss: 0.5737
Epoch 3/1000
988/988 - 18s - loss: 0.5586 - val_loss: 0.5422
Epoch 4/1000
988/988 - 18s - loss: 0.5475 - val_loss: 0.5387
Epoch 5/1000
988/988 - 18s - loss: 0.5410 - val_loss: 0.5307
Epoch 6/1000
988/988 - 18s - loss: 0.5363 - val_loss: 0.5280
Epoch 7/1000


In [ ]:
# best valid loss L7 0.53....52
# L3: 0.48 ish
# L3 bigger net: 0.48...0.47 with 12+14,
# after epoch 400 reduced lr to 0.00001
# 0.45 with 256 and 12+14, batch size 4096